In [58]:

import numpy as np
import argparse
import imutils #makes basix image processing resizing etc a lot easier
import sys
import cv2
import os
# pip install fer #if not already present
from fer import FER
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
from keras.utils.image_utils import  img_to_array
from keras.applications import xception

In [118]:
act_lst=open("Actions.txt").read().strip().split("\n")
#getting labels for all possible activities

In [97]:
# loading the kinetic model
md = cv2.dnn.readNet("resnet-34_kinetics.onnx")
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
emotion=FER(mtcnn=True)
weapon=tf.keras.models.load_model("weapon_detection.h5")
# accuracy above 90%
fire=tf.keras.models.load_model("fd.h5")
# accuracy near 90%
print(type(fire))

<class 'keras.engine.sequential.Sequential'>


In [130]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([0,0,250])
    upper_hsv = np.array([250,255,255])
    
    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

#image segmentation function
def segment_image(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#sharpen the image
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

def prepro(im):
    img=im.copy()
    img=cv2.resize(img,(255,255))
    img=img_to_array(img)
    image_segmented = segment_image(img)
    #sharpen
    image_sharpen = sharpen_image(image_segmented)
    x = xception.preprocess_input(np.expand_dims(image_sharpen.copy(), axis=0))
    return np.array(x)


In [30]:
def wpn(img):
    m,n=50,50
    im = img.copy()
    
    imrs = cv2.resize(img,(m,n))
    #print(type(imrs))
    imrs=img_to_array(imrs)/255
    imrs=imrs.transpose(2,0,1)
    imrs=imrs.reshape(3,m,n)
    x=np.array([imrs])
    predictions = weapon.predict(x)
    return predictions

In [95]:
# for weapon detection
label_names=sorted(['knife','no weapon'])
def preprocess(imge, image_size = 300):
    img=imge.copy()
    image = cv2.resize(img, (image_size, image_size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float") / 255.0
    # Expand dimensions as predict expect image in batches
    image = np.expand_dims(image, axis=0) 
    return image
def postprocess(img, results):
 
    # Split the results into class probabilities and box coordinates
    image=img.copy()
    bounding_box, class_probs = results
    class_index = np.argmax(class_probs)
   
    # Use this index to get the class name.
    class_label="No Weapon"
    if(class_index==0):
     class_label = "Weapon"

    h, w = image.shape[:2]
 
    # Extract the Coordinates
    x1, y1, x2, y2 = bounding_box[0]
 
    # Convert the coordinates from relative (i.e. 0-1) to actual values
    x1 = int(w * x1)
    x2 = int(w * x2)
    y1 = int(h * y1)
    y2 = int(h * y2)
 
    # return the lable and coordinates
    return class_label, (x1,y1,x2,y2),class_probs


In [81]:
xception_bf = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
def fir(img):
    im=img.copy()
    im=prepro(im)
    print(im.shape)
    xception_bf = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
   # bf_train_x = xception_bf.predict(train_x, batch_size=32, verbose=1)
    im = xception_bf.predict(im, batch_size=32, verbose=1)
    prd=fire.predict(im)
    return prd

In [133]:
cnt=0
cap=cv2.VideoCapture(0)
to_process=[]
original=[]
label="None"
while True:

    #now since we are video processing we would need continuous set of images instead of single frame by frame images we will need to convert it to blob 
    
    # lets take 16 frames at a time to make blob
   
    bl,img=cap.read()
    img=cv2.flip(img,1)
   # cv2.imshow('wnd',img)
    if bl==False:
        print("Unable to get input. Exiting code")
        cv2.destroyAllWindows()
        break

    
    x=imutils.resize(img,400)
    to_process.append(x)
    if(len(to_process)==16):
        blob=cv2.dnn.blobFromImages(to_process,1.0,(112,112), (114.7748, 107.7354, 99.4750),swapRB=True, crop=True)
        blob = np.transpose(blob, (1, 0, 2, 3))
        blob = np.expand_dims(blob, axis=0)
       # print(blob.shape)
        md.setInput(blob)
        ot=md.forward()
        label=act_lst[np.argmax(ot)]
        to_process.clear()

   
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    
   
   
    cnt=(cnt%16)
    s="None"
    # This part is commented out as my pc specs weren't good enough to compute multiple heavy models simultaneously
    # if(cnt==0):
    #     rt=fir(img)
    
    #     processed_image = preprocess(img)
    #     results = weapon.predict(processed_image)

    
    #     labe, (x1, y1, x2, y2), confidence = postprocess(img, results)
    #     if(labe=="Weapon"):
    #         cv2.rectangle(img, (x1,y1), (x2,y2), (0, 0, 255), 2)
    #         cv2.putText(
    #             img, 
    #             '{}'.format(labe, confidence), 
    #             (x1, y1-10), 
    #             cv2.FONT_HERSHEY_COMPLEX, 0.9,
    #             (0, 0, 255),2)
    #     if(rt[0][0]<0.5):
    #         cv2.putText(img,"Fire",(0,20),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)
    
    for (x, y, w, h) in faces_detected:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)
        dominant_emotion, emotion_score = emotion.top_emotion(img)
        cv2.putText(img, dominant_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.rectangle(img, (0, 0), (300, 40),
                    (0, 0, 0), -1)
    cv2.putText(img, label, (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (255, 255, 255), 2)
    cv2.imshow('wnd',img)
    cnt=cnt+1
    if cv2.waitKey(1) & 0XFF==ord('a'):
        break
cv2.destroyAllWindows()

In [123]:
#checking fire detection model on images
img=cv2.imread('fire.101.png')
rt=fir(img)
if(rt[0][0]>0.5):
    cv2.putText(img,"No Fire",(0,20),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)
else:
     cv2.putText(img,"Fire",(0,20),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)
cv2.imshow("le",img)
cv2.waitKey(10000)
cv2.destroyAllWindows()

(1, 255, 255, 3)
1/1 [==============================] - 0s 24ms/step


In [137]:
#checking knife detection on images
img=cv2.imread('11.jpeg')
processed_image = preprocess(img)
results = weapon.predict(processed_image)

labe, (x1, y1, x2, y2), confidence = postprocess(img, results)
print(labe)
cv2.rectangle(img, (x1,y1), (x2,y2), (0, 0, 255), 2)
cv2.putText(
    img, 
    '{}'.format(labe, confidence), 
    (x1, y1-10), 
    cv2.FONT_HERSHEY_COMPLEX, 0.9,
    (0, 0, 255),2)
cv2.imshow("le",img)
cv2.waitKey(10000)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 98ms/step
Weapon
